# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `clanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g., copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g., `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g., `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for some sequence containers (e.g., `vector` of the `std` namespace) and associative containers (e.g., `set`, `unordered_set` of the `std` namespace).
These template instantiations are done for various *C++* fundamental types (e.g., `int`, `unsigned long int`, `double`) and the `string` of the `std` namespace.
For ordered associative containers only the `std::less` comparator was used.
For the complete procedure refer to the `AutoWIG.py` file situed at the root of the **STL** [repository](https://github.com/StatisKit/STL).


We here aim at presenting how template libraries can be wrapped.
First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

On Windows OSes, the visual studio version used to compile future wrappers must be given.
But if the **SCons** tool is used, this version is known.

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [2]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

In this notebook, we do not need to import **AutoWIG** since **SCons** is configured to use the **Boost.Python** tool installed with **AutoWIG** that can be used to generate wrappers (see the `../git/STL/src/cpp/SConscript` file).

In [3]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\SConscript
else:
    !pygmentize ../git/STL/src/cpp/SConscript

# -*-python-*-

VECTORS = ['Index',
           'int',
           'double',
           'std::string']

SETS = { 'less': ['Index',
                  'int',
                  'double',
                  'std::string'],
         'none': ['Index']}

HEADER = """\
#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
   

The controller is registered in the `../git/STL/src/cpp/AutoWIG.py` file

In [4]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\AutoWIG.py
else:
    !pygmentize ../git/STL/src/cpp/AutoWIG.py

import autowig
import itertools

def controller(asg, **kwargs):
    autowig.controller.plugin = 'default'
    asg = autowig.controller(asg, **kwargs)
    for function in asg['::statiskit::stl'].functions():
        if function.localname in ['generator', 'insert']:
            parameter = function.parameters[0].qualified_type.desugared_type
            if parameter.is_class:
                function.parent = parameter.unqualified_type
    for cls in asg['class ::std::vector'].specializations(partial = False):
        for method in cls.methods():
            if method.localname in ['resize', 'shrink_to_fit', 'operator[]']:
                if isinstance(method.boost_python_export, bool):
                    method.boost_python_export = False
        for constructor in cls.constructors():
            if not(constructor.nb_parameters == 0 or constructor.nb_parameters == 1 and constructor.parameters[0].qualified_type.unqualified_type == cls):
                if isinstance(constructor.boost_p

Then, in addition to the **STL** library, the **StatisKit.STL** library has to be installed in order to have access to some functionalities.
To do so, we use available **Conda** recipes.

In [5]:
!conda remove libstatiskit_stl -y
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\libstatiskit_stl -c statiskit
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/libstatiskit_stl -c statiskit
!conda install -y libstatiskit_stl --use-local -c statiskit

Fetching package metadata .............
Solving package specifications: .

PackageNotFoundError: Package(s) is missing from the environment:
            [u'libstatiskit_stl
            

Attempting to finalize metadata for libstatiskit_stl
INFO:conda_build.metadata:Attempting to finalize metadata for libstatiskit_stl
BUILD START: [u'libstatiskit_stl-1.0.0-hee181fa_0.tar.bz2']

The following NEW packages will be INSTALLED:

    binutils_impl_linux-64: 2.28.1-h04c84fa_2             
    binutils_linux-64:      7.2.0-25                      
    ca-certificates:        2017.08.26-h1d4fec5_0         
    certifi:                2017.11.5-py27h71e7faf_0      
    gcc_impl_linux-64:      7.2.0-hc5ce805_2              
    gcc_linux-64:           7.2.0-25                      
    gfortran_impl_linux-64: 7.2.0-hb3c8cce_2              
    gfortran_linux-64:      7.2.0-25                      
    gxx_impl_linux-64:      7.2.0-hd3faf3d_2              
    gxx_linux-64:           7.2.0-25      

INFO: activate-gfortran_linux-64.sh made the following environmental changes:
+DEBUG_FFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/libstatiskit_stl_1516179902803/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold/include -fdebug-prefix-map=${SRC_DIR}=/usr/local/src/conda/${PKG_NAME}-${PKG_VERSION} -fdebug-prefix-map=${PREFIX}=/usr/local/src/conda-prefix -fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-all -fno-plt -Og -g -Wall -Wextra -fcheck=all -fbacktrace -fimplicit-none -fvar-tracking-assignments -pipe
+DEBUG_FORTRANFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/libstatiskit_stl_1516179902803/_h_env_placehold_pla

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations (see the `../git/STL/src/cpp/STL.h` file).

In [6]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\STL.h
else:
    !pygmentize ../git/STL/src/cpp/STL.h

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef unsigned long int Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generat

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **STL** library.
To do so, we need then to install the *C++* headers. 
This is done using the `cpp` target in **SCons**.

In [7]:
if is_windows:
    !scons cpp -C ..\git\STL
else:
    !scons cpp -C ../git/STL

scons: Entering directory `/home/pfernique/Desktop/devel/FP17/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/STL.h" as "/home/pfernique/.miniconda2/envs/fp17/include/statiskit/stl/STL.h"
/home/pfernique/.miniconda2/envs/fp17/bin/x86_64-conda_cos6-linux-gnu-g++ -o build/src/cpp/STL.os -c -std=c++11 -fvisibility=hidden -Wl,--no-undefined -fdiagnostics-color=always -fPIC -DLIBSTATISKIT_STL -I/home/pfernique/.miniconda2/envs/fp17/include build/src/cpp/STL.cpp
/home/pfernique/.miniconda2/envs/fp17/bin/x86_64-conda_cos6-linux-gnu-g++ -o /home/pfernique/.miniconda2/envs/fp17/lib/libstatiskit_stl.so -shared build/src/cpp/STL.os -L/home/pfernique/.miniconda2/envs/fp17/lib
scons: done building targets.


Once the headers have been installed in the system, we parse headers with relevant compilation flags.
This is done using the `autowig` target in **SCons**.

In [8]:
if is_windows:
    !scons autowig -C ..\git\STL
else:
    !scons autowig -C ../git/STL

scons: Entering directory `/home/pfernique/Desktop/devel/FP17/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/AutoWIG.py" as "/home/pfernique/.miniconda2/envs/fp17/lib/python2.7/site-packages/scons_tools/site_autowig/controller/statiskit_stl.py"
autowig: Generating Boost.Python interface ...
/home/pfernique/.miniconda2/envs/fp17/lib/python2.7/site-packages/clanglite/autowig_parser.py:844: Warning: 
  warnings.warn('', Warning)
scons: *** [/home/pfernique/.miniconda2/envs/fp17/lib/python2.7/site-packages/scons_tools/site_autowig/ASG/statiskit_stl.pkl] AttributeError : <class 'SCons.Node.FS.File'> object has no attribute 'startswith'
Traceback (most recent call last):
  File "/home/pfernique/.miniconda2/envs/fp17/lib/python2.7/site-packages/SCons/Action.py", line 1192, in execute
    result = self.execfunction(target=target, source=rsources, env=env)
  File "/home/pfernique/.miniconda2/envs/

Here is the list of the generated wrappers (untracked files).

In [9]:
if is_windows:
    !cd ..\git\STL & git status
else:
    !cd ../git/STL && git status

HEAD detached at 4756312
nothing to commit, working directory clean


And here, we present the wrappers generated for the `std::vector< int >` class.

In [10]:
if is_windows:
    !pygmentize ..\git\STL\src\py\wrapper\wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp
else:
    !pygmentize ../git/STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

Error: cannot read infile: [Errno 2] No such file or directory: '../git/STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp'


Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [11]:
!conda remove python-statiskit_stl -y
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\python-statiskit_stl -c statiskit
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/python-statiskit_stl -c statiskit
!conda install -y python-statiskit_stl --use-local -c statiskit

Fetching package metadata .............
Solving package specifications: .

PackageNotFoundError: Package(s) is missing from the environment:
            [u'python-statiskit_stl
            

Attempting to finalize metadata for python-statiskit_stl
INFO:conda_build.metadata:Attempting to finalize metadata for python-statiskit_stl
BUILD START: [u'python-statiskit_stl-3.3.1-py27ha3b329f_0.tar.bz2']

The following NEW packages will be INSTALLED:

    alabaster:                0.7.10-py27he5a193a_0         
    asn1crypto:               0.24.0-py27_0                 
    babel:                    2.5.1-py27_0                  
    binutils_impl_linux-64:   2.28.1-h04c84fa_2             
    binutils_linux-64:        7.2.0-25                      
    bzip2:                    1.0.6-h6d464ef_2              
    ca-certificates:          2017.08.26-h1d4fec5_0         
    certifi:                  2017.11.5-py27h71e7faf_0      
    cffi:                     1.11.2-py27ha7929c6_0         
    

INFO: activate-binutils_linux-64.sh made the following environmental changes:
+ADDR2LINE=/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516179965905/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/bin/x86_64-conda_cos6-linux-gnu-addr2line
+AR=/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516179965905/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/bin/x86_64-conda_cos6-linux-gnu-ar
+AS=/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516179965905/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/bin/x86_64-conda_cos6-linux-gnu-as
+CXXFILT=/home/pferniqu


scons py --prefix=$PREFIX -j$CPU_COUNT
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Creating 'build/src/py/wrapper/response_file.rsp'
Install file: "build/src/cpp/STL.h" as "/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516179965905/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/statiskit/stl/STL.h"
/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516179965905/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/bin/x86_64-conda_cos6-linux-gnu-g++ -o build/src/cpp/STL.os -c -std=c++11 -fvisibility=hidden -Wl,--no-undefined -fdiagnostics-color=always -fPIC -DLIBSTATISKIT_STL -I/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1

INFO: deactivate-gxx_linux-64.sh made the following environmental changes:
-CXXFLAGS=-fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe
-CXX=/home/pfernique/.miniconda2/envs/fp17/bin/x86_64-conda_cos6-linux-gnu-c++
-DEBUG_CXXFLAGS=-fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-all -fno-plt -Og -g -Wall -Wextra -fvar-tracking-assignments -pipe
-GXX=/home/pfernique/.miniconda2/envs/fp17/bin/x86_64-conda_cos6-linux-gnu-g++
INFO: deactivate-gcc_linux-64.sh made the following environmental changes:
-CC=/home/pfernique/.miniconda2/envs/fp17/bin/x86_64-conda_cos6-linux-gnu-cc
-CFLAGS=-march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe
-CPPFLAGS=-DNDEBUG -D_FORTIFY_SOURCE=2 -O2
-CPP=/home/pfernique/.miniconda2/envs/fp17/bin/x86_64-conda_cos6-linux-gnu-cpp
-DEBUG_CFLAGS=

INFO: activate-gfortran_linux-64.sh made the following environmental changes:
+DEBUG_FFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516179965905/_test_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include -fdebug-prefix-map=${SRC_DIR}=/usr/local/src/conda/${PKG_NAME}-${PKG_VERSION} -fdebug-prefix-map=${PREFIX}=/usr/local/src/conda-prefix -fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-all -fno-plt -Og -g -Wall -Wextra -fcheck=all -fbacktrace -fimplicit-none -fvar-tracking-assignments -pipe
+DEBUG_FORTRANFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1516179965905/_test_env_placeh

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [12]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

ImportError: No module named statiskit.stl

In [ ]:
list(v)

In [ ]:
v[0]

In [ ]:
v[0] = -2
v[0]

In [ ]:
VectorInt([0, 1])

Here is a report concerning objects wrapped using this notebook.

In [ ]:
import fp17
import os
import pickle
with open(os.path.join(os.environ['SITE_SCONS'],
                       'site_autowig',
                       'ASG',
                       'statiskit_stl.pkl'), 'rb') as filehandler:
    asg = pickle.load(filehandler)
fp17.report(asg)